In [ ]:
import pandas as pd
import numpy as np

In [ ]:
date_format = '%d.%m.%y %H:%M'

for i in range(1, 29):
    file_path = 'data/raw/subjectID_' + str(i) + '.csv'
    df = pd.read_csv(file_path)
    # if there is an acitivity class column, drop it
    if 'ActivityClass' in df.columns:
        df = df.drop(['ActivityClass'], axis=1)
    # if there is a galvanic skin response column, drop it
    if 'GalvanicSkinResponse' in df.columns:
        df = df.drop(['GalvanicSkinResponse'], axis=1)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format=date_format)
    print('subjectID_' + str(i) + ':')
    print('The number of weeks that the Timestamp column spans:')
    print(df['Timestamp'].max() - df['Timestamp'].min())
    print('The number of rows with missing values, and their percentage of the total number of rows:')
    print(df.isnull().sum() / df.shape[0] * 100)
    print('\n')
